In [1]:
!python -V

Python 3.8.13


In [17]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import mlflow
import os

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [18]:
TRACK_DIR = os.path.abspath('.')
INTRO_DIR = os.path.join(os.path.dirname(TRACK_DIR), '01-intro')
DATA_DIR = os.path.join(INTRO_DIR, 'data')

Initial monitoring experiments

In [19]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='./mlruns/1', creation_time=1675888710859, experiment_id='1', last_update_time=1675888710859, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [20]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [5]:
df_train = read_dataframe(os.path.join(DATA_DIR, 'green_tripdata_2021-01.parquet'))
df_val = read_dataframe(os.path.join(DATA_DIR, 'green_tripdata_2021-02.parquet'))

In [6]:
len(df_train), len(df_val)

(73908, 61921)

In [22]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [23]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [24]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [25]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.7587151974993125

In [11]:
model_dir = os.path.join(TRACK_DIR, 'models')
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    with open(os.path.join(model_dir, 'lin_reg.bin'), 'wb') as f_out:
        pickle.dump((dv, lr), f_out)

In [12]:
model_dir = os.path.join(TRACK_DIR, 'models')
train_data_path = os.path.join(DATA_DIR, 'green_tripdata_2021-01.parquet')
val_data_path = os.path.join(DATA_DIR, 'green_tripdata_2021-02.parquet')

with mlflow.start_run():

    mlflow.set_tag('developer', 'weslley')

    # add data path used for  training and validation
    mlflow.log_param('train-data-path', train_data_path)
    mlflow.log_param('valid-data-path', val_data_path)

    alpha = 0.01
    mlflow.log_param('alpha', alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)

    mlflow.log_artifact(local_path=model_dir, artifact_path='models_pickle')

### Hyper Params Tuning

In [26]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [27]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [28]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [16]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[16:20:32] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.89607                          
[1]	validation-rmse:18.70398                          
[2]	validation-rmse:17.60801                          
[3]	validation-rmse:16.60174                          
[4]	validation-rmse:15.67907                          
[5]	validation-rmse:14.83408                          
[6]	validation-rmse:14.06051                          
[7]	validation-rmse:13.35370                          
[8]	validation-rmse:12.70939                          
[9]	validation-rmse:12.12243                          
[10]	validation-rmse:11.59027                         
[11]	validation-rmse:11.10551                         
[12]	validation-rmse:10.66643                         
[13]	validation-rmse:10.26790                         
[14]	validation-rmse:9.90955                          
[15]	validation-rmse:9.58489                          
[

KeyboardInterrupt: 

In [18]:
mlflow.xgboost.autolog(disable=True)

In [29]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.17850613380437869,
        'max_depth': 28,
        'min_child_weight': 1.1270192461644135,
        'objective': 'reg:linear',
        'reg_alpha': 0.012914472583733764,
        'reg_lambda': 0.010271275385850088,
        'seed': 42
    }

    mlflow.log_params(best_params)

    mlflow.xgboost.autolog()

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open(os.path.join(model_dir, 'preprocessor.b'), "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact(os.path.join(model_dir, 'preprocessor.b'), artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

2023/02/09 22:09:39 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of xgboost. If you encounter errors during autologging, try upgrading / downgrading xgboost to a supported version, or try upgrading MLflow.


[22:09:39] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:18.02731
[1]	validation-rmse:15.48991
[2]	validation-rmse:13.46727
[3]	validation-rmse:11.86676
[4]	validation-rmse:10.61678
[5]	validation-rmse:9.64692
[6]	validation-rmse:8.90328
[7]	validation-rmse:8.33543
[8]	validation-rmse:7.90723
[9]	validation-rmse:7.58181
[10]	validation-rmse:7.33788
[11]	validation-rmse:7.15217
[12]	validation-rmse:7.00981
[13]	validation-rmse:6.90078


KeyboardInterrupt: 

In [23]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/Users/cristian.martinez/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
